In [12]:
# Je dois un peut travailler dans un notebook 

In [13]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

In [14]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim

In [15]:
torch.__version__

'2.1.1+cpu'

Creation du DataSet

In [16]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize
])

image_datasets = datasets.ImageFolder(
    "./data/",
    transform
)

dataloader = torch.utils.data.DataLoader(image_datasets,
                                batch_size=32,
                                shuffle=True, num_workers=4),


In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

device

device(type='cpu')

In [18]:
model = models.resnet50(pretrained=True).to(device)

for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, len(image_datasets))).to(device)

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [27]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        model.train()

        running_loss = 0.0
        running_corrects = 0

        for _, datas in enumerate(dataloader):
            inputs, labels = datas
            print(datas)
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, 1)
            running_loss += loss.detach() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(image_datasets)
        epoch_acc = running_corrects.float() / len(image_datasets)

        print('loss: {:.4f}, acc: {:.4f}'.format(epoch_loss.item(),
                                                    epoch_acc.item()))
    return model

In [29]:
# model_trained = train_model(model, criterion, optimizer, num_epochs=3)
print(dataloader)
for i, l in dataloader:
    print(i)

(<torch.utils.data.dataloader.DataLoader object at 0x000001D21EE515D0>,)
